<a href="https://colab.research.google.com/github/invictus125/cs598-final-project/blob/main/intraoperative_hypotension.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A Reproduction of:
## Predicting intraoperative hypotension using deep learning with waveforms of arterial blood pressure, electroencephalogram, and electrocardiogram

Original paper by: Yong-Yeon Jo, Jong-Hwan Jang, Joon-myoung Kwon, Hyung-Chul Lee, Chul-Woo Jung, Seonjeong Byun, Han‐Gil Jeong

Reproduction project authored by
* Mark Bauer
  * mbauer553
  * markab5@illinois.edu
* Ryan David
  * victheone
  * invictus125
  * radavid2@illinois.edu

This project can be found on github https://github.com/invictus125/cs598-final-project.  

> Note that this project uses <b>VitalDB, an open biosignal dataset.  All users must agree to the Data Use Agreement below.</b>  If after reviewing the agreement you do not comply, please do not read on and close this window.
[Data Use Agreement](https://vitaldb.net/dataset/?query=overview&documentId=13qqajnNZzkN7NZ9aXnaQ-47NWy7kx-a6gbrcEsi-gak&sectionId=h.vcpgs1yemdb5)

## Introduction
Our project is to perform an approximate reproduction of a paper, which can be found [here](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0272055), on predicting hypotension during surgery from a combination of signals such as mean arterial blood pressure (ABP), electrocardiogram (ECG), and electroencephalogram (EEG) as opposed to ABP alone.  Predicting hypotension is important because it is correlated with many post operation complications, and is actionable.  Please read the original work if you are interested in more detail!

## Scope of reproducibility

As of the draft on 2024-04-14, we are using what we understand to be a very close replication of the model, with a smaller openly available data set.  Our model currently executes utilizing ABP data from the dataset for predictors and labels, but the EEG and ECG data is randomly created mock data.  The original work also examined more look ahead times.  

In [1]:
# install dependencies
!pip install torcheval
!pip install vitaldb

# mount drive
from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.0/160.0 kB 9.7 MB/s eta 0:00:00
Mounted at /content/drive


## Methodology - Data

Methodology - Data
Case: a surgery/operation

Track: data observed during a case, consisting of a device and type

As of the draft on 2024-04-14 we are getting only ABP data and labels looking ahead one minute.  

In [2]:
from functools import partial
from io import StringIO
from os import listdir
from torch import FloatTensor, BoolTensor

import csv
import numpy as np
import requests
import vitaldb

SEGEMENT_LENGTH_SECONDS = 60

ABP_TRACK = 'SNUADC/ART'
ECG_TRACK = 'SNUADC/ECG_II'
EEG_TRACK = 'BIS/EEG1_WAV'

RELEVANT_TRACKS = [
    ABP_TRACK,
    ECG_TRACK,
    EEG_TRACK,
]

def _url_to_reader(url_string):
    response = requests.get(url_string)
    file = StringIO(response.text)
    return csv.DictReader(file, delimiter=',')

def get_unique_vals(key, iterable):
    return set(map(lambda item: item[key], iterable))

def case_filter(case):
    return float(case['age']) >= 18.0 and case['ane_type'] == 'General'

def _case_track_filter(case_id, case_dict):
    track_list = case_dict[case_id]['tracks']
    return (
        ABP_TRACK in track_list and
        ECG_TRACK in track_list and
        EEG_TRACK in track_list
    )

def _get_candidate_cases():
    cases_by_id = {}
    for case in _url_to_reader('https://api.vitaldb.net/cases'):
        if case_filter(case):
            case['tracks'] = {}
            cases_by_id[case['\ufeffcaseid']] = case

    track_list_reader = _url_to_reader('https://api.vitaldb.net/trks')

    for track in track_list_reader:
        case_id = track['caseid']
        if track['tname'] in RELEVANT_TRACKS:
            if cases_by_id.get(case_id):
                cases_by_id[case_id]['tracks'][track['tname']] = track['tid']

    case_track_filter = partial(_case_track_filter, case_dict=cases_by_id)

    return [case_id for case_id in filter(case_track_filter, cases_by_id.keys())]

def _get_candidate_cases_from_dir(dir_path):
    return [f.split('.vital')[0] for f in listdir(dir_path) if '.vital' in f]

def _download_vital_file(case_id):
    vf = vitaldb.VitalFile(int(case_id), RELEVANT_TRACKS)
    vf.to_vital(case_id+'.vital')

def _get_tracks_from_vital_file(path, tracks, sample_rate):
    vf = vitaldb.read_vital(path, tracks)
    return vf.to_numpy(tracks, sample_rate)


def shuffle_tensors(tensors):
  shuffled_output = []
  shuffled_index = np.array([i for i in range(0, len(tensors[0]))]).astype(int)
  np.random.shuffle(shuffled_index)
  for t in tensors:
    if len(t) > 0:
      new_tensor = t.detach().clone()[shuffled_index]
      shuffled_output.append(new_tensor.squeeze(0))
    else:
      shuffled_output.append(t)

  return shuffled_output


def validate_abp_segment(segment):

    return (
        not np.isnan(segment).any() and
        not (segment > 200).any() and
        not (segment < 30).any() and
        not ((np.max(segment) - np.min(segment)) < 30) and
        not (np.abs(np.diff(segment)) > 30).any() # abrupt changes are assumed to be noise
    )

def download_data(num_requested_cases):
    num_downloaded_cases = 0
    candidate_case_ids = _get_candidate_cases()

    np.random.shuffle(candidate_case_ids)
    for case_id in candidate_case_ids:
        print('Downloading case:', case_id)
        _download_vital_file(case_id)
        num_downloaded_cases = num_downloaded_cases + 1
        at_requested = num_downloaded_cases == num_requested_cases
        if at_requested:
            break

    if not at_requested:
        print('Requsted cases not reached but all available cases exhausted.  ')

def get_data(
    minutes_ahead,
    abp_and_ecg_sample_rate_per_second=500,
    eeg_sample_rate_per_second=128,
    case_ids=None,
    max_num_samples=None,
    max_num_cases=None,
    from_dir=None,
    shuffle_samples=False,
):
    if case_ids is not None:
        candidate_case_ids = case_ids
    elif from_dir is None:
        candidate_case_ids = _get_candidate_cases()
    else:
        candidate_case_ids = _get_candidate_cases_from_dir(from_dir)

    abps = []
    ecgs = []
    eegs = []
    hypotension_event_bools = []

    abp_data_in_two_seconds = 2 * abp_and_ecg_sample_rate_per_second

    at_max = False

    case_count = 0
    np.random.shuffle(candidate_case_ids)
    for case_id in candidate_case_ids:
        case_num_samples = 0
        case_num_events = 0

        print('Getting track data for case:', case_id)
        if from_dir is None:
            case_tracks = vitaldb.load_case(int(case_id), RELEVANT_TRACKS[0:2], 1/abp_and_ecg_sample_rate_per_second)
        else:
            case_tracks = _get_tracks_from_vital_file(f"{from_dir}/{case_id}.vital", RELEVANT_TRACKS[0:2], 1/abp_and_ecg_sample_rate_per_second)

        abp_track = case_tracks[:,0]
        # ecg_track = case_tracks[:,1]

        # eeg_track = vitaldb.load_case(int(case_id), RELEVANT_TRACKS[2], 1/eeg_sample_rate_per_second).flatten()

        for i in range(
            0,
            len(abp_track) - abp_and_ecg_sample_rate_per_second * (SEGEMENT_LENGTH_SECONDS + (1 + minutes_ahead) * SEGEMENT_LENGTH_SECONDS),
            10 * abp_and_ecg_sample_rate_per_second
        ):
            x_segment = abp_track[i:i + abp_and_ecg_sample_rate_per_second * SEGEMENT_LENGTH_SECONDS]
            y_segment_start = i + abp_and_ecg_sample_rate_per_second * (SEGEMENT_LENGTH_SECONDS + minutes_ahead * SEGEMENT_LENGTH_SECONDS)
            y_segement_end = i + abp_and_ecg_sample_rate_per_second * (SEGEMENT_LENGTH_SECONDS + (minutes_ahead + 1) * SEGEMENT_LENGTH_SECONDS)
            y_segment = abp_track[y_segment_start:y_segement_end]

            if validate_abp_segment(x_segment) and validate_abp_segment(y_segment):
                abps.append(x_segment)

                # 2 second moving average
                y_numerator = np.nancumsum(y_segment, dtype=np.float32)
                y_numerator[abp_data_in_two_seconds:] = y_numerator[abp_data_in_two_seconds:] - y_numerator[:-abp_data_in_two_seconds]
                y_moving_avg = y_numerator[abp_data_in_two_seconds - 1:] / abp_data_in_two_seconds

                is_hypotension_event = np.nanmax(y_moving_avg) < 65
                hypotension_event_bools.append(is_hypotension_event)
                case_num_samples = case_num_samples + 1
                if(is_hypotension_event):
                    case_num_events = case_num_events + 1

            at_max_samples = len(hypotension_event_bools) == max_num_samples
            if at_max_samples:
                break

        case_count = case_count + 1
        print(f"Statistics for case: {case_id}, {case_num_samples} total valid samples, {case_num_events} positive samples")

        if at_max_samples or case_count == max_num_cases:
            if at_max_samples:
                print('Max samples reached')
            else:
                print('Max cases reached')
            at_max = True
            break

    if (max_num_samples is not None or max_num_cases is not None) and not at_max:
        print('Max not reached but all available cases exhausted.  ')

    print('Converting and shuffling (if enabled)')

    abps = FloatTensor(abps)
    ecgs = FloatTensor(ecgs)
    eegs = FloatTensor(eegs)
    hypotension_event_bools = BoolTensor(hypotension_event_bools).float()

    if shuffle_samples:
      output_tensors = shuffle_tensors([abps, ecgs, eegs, hypotension_event_bools])
    else:
      output_tensors = [abps, ecgs, eegs, hypotension_event_bools]

    return (
        output_tensors[0].unsqueeze(1),
        output_tensors[1].unsqueeze(1),
        output_tensors[2].unsqueeze(1),
        output_tensors[3]
    )


def _process_loaded_abp_data(all_data, negative_ratio=1.0):
  pos_indices = torch.nonzero(all_data[3] > 0.5).squeeze(-1)
  neg_indices = torch.nonzero(all_data[3] < 0.5).squeeze(-1)
  abp_pos = all_data[0].detach().clone()[pos_indices]
  abp_neg = all_data[0].detach().clone()[neg_indices]

  positive_samples = len(pos_indices)
  cutoff = floor(positive_samples * negative_ratio)
  print(f'block has {positive_samples} positives')
  abp_neg = abp_neg[0:cutoff]

  all_abp = torch.concat([abp_neg, abp_pos])
  all_labels = torch.concat([torch.zeros([abp_neg.size()[0]]), torch.ones([abp_pos.size()[0]])])
  shuffled = shuffle_tensors([all_abp, all_labels])
  return [
    shuffled[0],
    torch.Tensor([]),
    torch.Tensor([]),
    shuffled[1]
  ]


## Methodology - Model
Our model is an exact reproduction based on the description provided in the original paper.

There is a ResNet for each of the three waveform types we handle consisting of:

- A CNN encoder layer
- 12 residual blocks, each having two convolutions and two batch normalizations. Alternating blocks will halve the length of the data using a max pooling operation. Per the paper, we also added skip connections by summing the input into the output in each residual block.
- A fully connected output layer which flattens the channels prior to passing through a NN

The model is built such that we can provide one or more ResNets and it will adapt. This is so that we can experiment with varying combinations of input data.

Once the input is run through the ResNets, their output is concatenated and passed through a fully connected layer which ends with a sigmoid activation, producing the final prediction.


In [3]:
import torch
import math
from torch import nn


class EncoderBlock(nn.Module):
  def __init__(self, dim_in, kernel_size=15, stride=1):
    super(EncoderBlock, self).__init__()
    padding = math.floor(kernel_size / 2.0)
    self.conv = nn.Conv1d(1, 1, kernel_size, stride, padding=padding)
    self.mp = nn.MaxPool1d(kernel_size, stride, padding)
    self.fc = nn.Linear(dim_in, dim_in)
    torch.nn.init.normal_(self.fc.weight, mean=0.0, std=0.01)


  def forward(self, x):
    x_hat = self.conv(x)
    x_hat = self.mp(x_hat)
    return self.fc(x_hat)


class ResidualBlock(nn.Module):
  def __init__(
    self,
    in_channels,
    out_channels,
    size_down,
    kernel_size,
    stride=1
  ):
    super(ResidualBlock, self).__init__()

    self.size_down = size_down
    self.in_channels = in_channels
    self.out_channels = out_channels

    padding = math.floor(kernel_size / 2.0)

    self.bn1 = nn.BatchNorm1d(in_channels)
    self.act1 = nn.ReLU()
    self.do = nn.Dropout()
    self.conv1 = nn.Conv1d(in_channels, in_channels, kernel_size, stride, padding)
    self.bn2 = nn.BatchNorm1d(in_channels)
    self.act2 = nn.ReLU()
    self.conv2 = nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding)
    self.mp = nn.MaxPool1d(kernel_size, padding=padding, stride=2)
    self.conv_for_input = nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding)


  def forward(self, x):
    x_hat = self.bn1(x)
    x_hat = self.act1(x_hat)
    x_hat = self.do(x_hat)
    x_hat = self.conv1(x_hat)
    x_hat = self.bn2(x_hat)
    x_hat = self.act2(x_hat)
    x_hat = self.conv2(x_hat)

    # Adjust dimensions of input if needed for the skip connection
    x_input = None
    if self.in_channels != self.out_channels:
      x_input = self.conv_for_input(x)
    else:
      x_input = x

    x_hat = x_hat + x_input

    if self.size_down:
      x_hat = self.mp(x_hat)

    return x_hat


class FlattenAndLinearBlock(nn.Module):
  def __init__(self, dim_in, dim_out):
    super(FlattenAndLinearBlock, self).__init__()
    self.fc = nn.Linear(dim_in, dim_out)
    torch.nn.init.normal_(self.fc.weight, mean=0.0, std=0.01)


  def forward(self, x):
    x_hat = torch.flatten(x, start_dim=1, end_dim=-1)
    x_hat = self.fc(x_hat)

    return x_hat


class WaveformResNet(nn.Module):
  def __init__(
    self,
    input_shape,
    output_size,
    data_type
  ):
    super(WaveformResNet, self).__init__()
    self.encoder = EncoderBlock(input_shape, 15, 1)
    self.res_in_dim = input_shape
    self.output_size = output_size
    self.data_type = data_type

    if data_type not in ['abp', 'ecg', 'eeg']:
      raise ValueError('Invalid data type. Must be one of [abp, ecg, eeg]')

    # Set up configurations for residual blocks
    residual_configs = []
    linear_block_input_length = -1
    if data_type in ['abp', 'ecg']:
      residual_configs = [
        {
          'kernel_size': 15,
          'in_channels': 1,
          'out_channels': 2,
          'size_down': True,
        },
        {
          'kernel_size': 15,
          'in_channels': 2,
          'out_channels': 2,
          'size_down': False,
        },
        {
          'kernel_size': 15,
          'in_channels': 2,
          'out_channels': 2,
          'size_down': True,
        },
        {
          'kernel_size': 15,
          'in_channels': 2,
          'out_channels': 2,
          'size_down': False,
        },
        {
          'kernel_size': 15,
          'in_channels': 2,
          'out_channels': 2,
          'size_down': True,
        },
        {
          'kernel_size': 15,
          'in_channels': 2,
          'out_channels': 4,
          'size_down': False,
        },
        {
          'kernel_size': 7,
          'in_channels': 4,
          'out_channels': 4,
          'size_down': True,
        },
        {
          'kernel_size': 7,
          'in_channels': 4,
          'out_channels': 4,
          'size_down': False,
        },
        {
          'kernel_size': 7,
          'in_channels': 4,
          'out_channels': 4,
          'size_down': True,
        },
        {
          'kernel_size': 7,
          'in_channels': 4,
          'out_channels': 6,
          'size_down': False,
        },
        {
          'kernel_size': 7,
          'in_channels': 6,
          'out_channels': 6,
          'size_down': True,
        },
        {
          'kernel_size': 7,
          'in_channels': 6,
          'out_channels': 6,
          'size_down': False,
        },
      ]
      linear_block_input_length = 469 * 6
    else:
      residual_configs = [
        {
          'kernel_size': 7,
          'in_channels': 1,
          'out_channels': 2,
          'size_down': True,
        },
        {
          'kernel_size': 7,
          'in_channels': 2,
          'out_channels': 2,
          'size_down': False,
        },
        {
          'kernel_size': 7,
          'in_channels': 2,
          'out_channels': 2,
          'size_down': True,
        },
        {
          'kernel_size': 7,
          'in_channels': 2,
          'out_channels': 2,
          'size_down': False,
        },
        {
          'kernel_size': 7,
          'in_channels': 2,
          'out_channels': 2,
          'size_down': True,
        },
        {
          'kernel_size': 7,
          'in_channels': 2,
          'out_channels': 4,
          'size_down': False,
        },
        {
          'kernel_size': 3,
          'in_channels': 4,
          'out_channels': 4,
          'size_down': True,
        },
        {
          'kernel_size': 3,
          'in_channels': 4,
          'out_channels': 4,
          'size_down': False,
        },
        {
          'kernel_size': 3,
          'in_channels': 4,
          'out_channels': 4,
          'size_down': True,
        },
        {
          'kernel_size': 3,
          'in_channels': 4,
          'out_channels': 6,
          'size_down': False,
        },
        {
          'kernel_size': 3,
          'in_channels': 6,
          'out_channels': 6,
          'size_down': True,
        },
        {
          'kernel_size': 3,
          'in_channels': 6,
          'out_channels': 6,
          'size_down': False,
        },
      ]
      linear_block_input_length = 120 * 6

    self.residuals = []
    # Build residuals
    for i in range(12):
      self.residuals.append(
        ResidualBlock(
          size_down=residual_configs[i]['size_down'],
          in_channels=residual_configs[i]['in_channels'],
          out_channels=residual_configs[i]['out_channels'],
          kernel_size=residual_configs[i]['kernel_size'],
        )
      )

    self.fl_ln = FlattenAndLinearBlock(linear_block_input_length, output_size)


  def forward(self, x):
    # x_hat = self.encoder(x)
    x_hat = x

    for i in range(len(self.residuals)):
      x_hat = self.residuals[i](x_hat)

    out = self.fl_ln(x_hat)

    return out


  def get_output_size(self):
    return self.output_size


class IntraoperativeHypotensionModel(nn.Module):
  def __init__(
    self,
    ecg_resnet=None,
    abp_resnet=None,
    eeg_resnet=None
  ):
    super(IntraoperativeHypotensionModel, self).__init__()

    self.ecg = ecg_resnet
    self.abp = abp_resnet
    self.eeg = eeg_resnet

    self.fc_input_length = 0

    if self.ecg is not None:
      self.fc_input_length += self.ecg.get_output_size()

    if self.abp is not None:
      self.fc_input_length += self.abp.get_output_size()

    if self.eeg is not None:
      self.fc_input_length += self.eeg.get_output_size()

    if self.fc_input_length == 0:
      raise 'No resnet blocks provided, unable to build model'

    self.fc1 = nn.Linear(self.fc_input_length, 16)
    self.fc2 = nn.Linear(16, 1)
    self.act = nn.Sigmoid()
    torch.nn.init.normal_(self.fc1.weight, mean=0.0, std=0.01)
    torch.nn.init.normal_(self.fc2.weight, mean=0.0, std=0.01)


  def forward(self, abp, ecg, eeg):
    ecg_o = torch.Tensor([])
    abp_o = torch.Tensor([])
    eeg_o = torch.Tensor([])

    if self.ecg is not None:
      ecg_o = self.ecg(ecg)

    if self.abp is not None:
      abp_o = self.abp(abp)

    if self.eeg is not None:
      eeg_o = self.eeg(eeg)

    resnet_output = torch.concat([ecg_o, abp_o, eeg_o], dim=1)

    intermediate = self.fc1(resnet_output)
    intermediate = self.fc2(intermediate)

    prediction = self.act(intermediate)

    return prediction


## Methodology - Training
Computational requirements:
- At least 50 GB of RAM
- GPU instance (we have been experimenting with an A100)

The training of this model is fairly straightforward. The paper suggested that we should use Adam as the optimizer and BCE as the loss function, so that is what we have done.

Each training epoch will also automatically run evaluation on both the train set and the validation set. See the evaluation methodology for details.

In [4]:
import torch
from torch.optim import Adam
from torch.nn import BCELoss
import numpy as np
import pickle
import time


def _extract_batch(data, batch_size, batch_number):
  start = batch_size * batch_number
  end = start + batch_size

  if start >= len(data[3]):
    return None

  return [
      data[0][start:end] if len(data[0]) > 0 else None,
      data[1][start:end] if len(data[1]) > 0 else None,
      data[2][start:end] if len(data[2]) > 0 else None,
      data[3][start:end]
  ]


def _write_perf_checkpoint(out_dir, epoch, model_title, performance):
  dir = f'{out_dir}/{model_title}/perf'
  !mkdir -p {dir}
  perf_path = f'{dir}/{epoch}_perf.txt'
  perf_file = open(perf_path, 'w')
  perf_file.write(f'AUROC: {performance[0]}\nAUPRC: {performance[1]}\nSensitivity: {performance[2]}\nSpecificity: {performance[3]}')
  perf_file.close()


def _write_model_checkpoint(model, out_dir, epoch, model_title):
  dir = f'{out_dir}/{model_title}/{epoch}'
  !mkdir -p {dir}
  model_path = f'{dir}/model'
  torch.save(model.state_dict(), model_path)


def _train_one_epoch(
  model,
  train_data,
  optimizer,
  batch_size=32,
  pos_weight=0.9,
  neg_weight=0.1,
  device
):
  start = time.time()
  model.train()
  loss_history = []

  batch_num = 0
  batch = _extract_batch(train_data, batch_size, batch_num)

  while batch is not None:
    # Build weight tensor and criterion function
    weight = batch[3].detach().clone()
    weight = torch.where(weight > 0.5, pos_weight, neg_weight)
    criterion = BCELoss(weight=weight)

    optimizer.zero_grad()
    abp = batch[0].to(device)
    ecg = batch[1].to(device)
    eeg = batch[2].to(device)
    y = batch[3].to(device)
    y_hat = model(abp, ecg, eeg)
    y_hat = torch.squeeze(y_hat, dim=-1)
    loss = criterion(y_hat, y)
    loss.backward()
    optimizer.step()
    batch_loss = loss.item()
    # print(f'\tBatch {batch_num} loss: {batch_loss}')
    loss_history.append(batch_loss)

    batch_num += 1
    batch = _extract_batch(train_data, batch_size, batch_num)

  end = time.time()

  print(f'Elapsed train time for batch: {end - start}')

  return loss_history


def train(
  model,
  train_data_handle,
  test_data_handle,
  learning_rate=0.0001,
  epochs=100,
  suspend_train_epochs_threshold=5,
  batch_size=32,
  output_dir=None,
  model_title='ioht_model'
):
  """Trains an IntraoperativeHypotensionModel using the given learning rate for
  the given number of epochs

  model: the IntraoperativeHypotensionModel to train
  train_data_handle: the dataset we will train on
  test_data_handle: the dataset we will use for evaluation
  learning_rate: the learning rate to use with the Adam optimizer
  epochs: the number of epochs to train for
  suspend_train_epochs_threshold: training will be suspended if the loss does
    not improve for this number of epochs
  batch_size: the size of batches to train on
  output_dir: the path to an output directory
  model_title: the title of the model to use when writing checkpoints
  """
  if model is None or train_data_handle is None or test_data_handle is None:
    raise ValueError(
      'model, train_data_handle, and test_data_handle are required for training'
    )

  # Determine if we can use CUDA / GPU training
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

  # Put the model onto the correct device prior to training
  model.to(device)

  optimizer = Adam(model.parameters(), lr=learning_rate)

  overall_loss_history = []
  consecutive_epochs_without_improvement = 0
  for epoch in range(epochs):
    print('====================================')
    print(f'     Epoch #{epoch + 1}')
    print('====================================')

    loss_history = _train_one_epoch(
      model,
      train_data_handle,
      optimizer,
      batch_size
    )
    eval_model(model, train_data_handle, 'Train', batch_size)

    performance = eval_model(model, test_data_handle, 'Test', batch_size)

    _write_perf_checkpoint(
      output_dir,
      epoch,
      model_title,
      performance
    )

    if output_dir is not None and (epoch % 5) == 0:
      # Write a model checkpoint every 5 epochs (model checkpoints are large)
      _write_model_checkpoint(
        model,
        output_dir,
        epoch,
        model_title
      )

    if epoch > 0:
      mean_loss = np.mean(loss_history)
      overall_loss_history.append(mean_loss)
      print(f'Mean loss: {mean_loss}')


##  Methodology - Evaluation
The original paper uses four metrics:
- AUROC
- AUPRC
- Sensitivity
- Specificity

We chose to use the torcheval library for our metrics, except for binary specificity which did not appear to be present in torcheval.

In light of that, we have implemented our own binary specificity. Regrettably, this does not yet work properly, but we will do our best to get it functional before our final submission.



In [5]:
import torch
from torcheval.metrics import BinaryAUROC, BinaryAUPRC, BinaryRecall


def _binary_specificity(test, target):
  # TN / TN + FP
  pinned = torch.where(test >= 0.5, 1.0, 0.0)
  pos = torch.where(pinned > 0, 1.0, 0.0)
  neg = torch.where(pinned < 1, 1.0, 0.0)
  gt_pos = torch.where(target > 0, 1.0, 0.0)
  gt_neg = torch.where(target < 1, 1.0, 0.0)
  tn = neg + gt_neg
  tn = torch.sum(torch.where(tn > 1, 1.0, 0.0), dtype=torch.float)
  fp = pos + gt_neg
  fp = torch.sum(torch.where(fp > 1, 1.0, 0.0), dtype=torch.float)

  return (tn / (tn + fp))


def eval_model(
  model,
  eval_data,
  dataset_name,
  batch_size=32,
  device
):
  # Reduce memory usage during eval
  with torch.no_grad():
    model.eval()

    # auroc = []
    # auprc = []
    # sensitivity = []
    specificity = []

    f_auroc = BinaryAUROC()
    f_auprc = BinaryAUPRC()
    f_sensitivity = BinaryRecall()

    batch_num = 0
    batch = _extract_batch(eval_data, batch_size, batch_num)

    while batch is not None:
      abp = batch[0].to(device)
      ecg = batch[1].to(device)
      eeg = batch[2].to(device)
      y = batch[3].to(device)

      y_hat = model(abp, ecg, eeg)
      y_hat = y_hat.squeeze(-1)

      # y_hat_long = torch.where(y_hat >= 0.5, 1.0, 0.0).long()
      # target_long = y.long()
      # print(f'y_hat_long sum: {y_hat_long.sum()}, target_long sum: {target_long.sum()}')

      f_auroc.update(y_hat, y)
      f_auprc.update(y_hat, y)
      f_sensitivity.update(y_hat_long, target_long)

      # auroc.append(f_auroc.compute())
      # auprc.append(f_auprc.compute())
      # sensitivity.append(f_sensitivity.compute())
      specificity.append(_binary_specificity(y_hat, y))

      batch_num += 1
      batch = _extract_batch(eval_data, batch_size, batch_num)

    # m_auroc = np.mean(auroc)
    # m_auprc = np.mean(auprc)
    # m_sensitivity = np.mean(sensitivity)
    # m_specificity = np.mean(specificity)
    m_auroc = f_auroc.compute()
    m_auprc = f_auprc.compute()
    m_sensitivity = f_sensitivity.compute()
    m_specificity = np.mean(specificity)

    print(f'    {dataset_name} data metrics:')
    print(f'        AUROC: {m_auroc}')
    print(f'        AUPRC: {m_auprc}')
    print(f'        Sensitivity: {m_sensitivity}')
    print(f'        Specificity: {m_specificity}')

    return (m_auroc, m_auprc, m_sensitivity, m_specificity)


### Results
Thus far we have the ability to build our model and obtain one type of test data in the form we need it for training and evaluation. Please execute the code cells below to see for yourself!

Our plans from here until the end of the project are:

- Make it possible to obtain the other two data types (ECG and EEG) and use them the same way we are able to use ABP
- Get our custom specificity function working properly
- Train and evaluate our model using a variety of samples
- Train and evaluate our model using varying combinations of the input types


# Train with only ABP

This next section will demonstrate training on only ABP data. It also demonstrates the scalability of our model, allowing us to only use certain resnets if we so choose.

In [6]:
# Put together a model using only ABP
abp_resnet = WaveformResNet(
    input_shape=30000,
    output_size=32,
    data_type='abp',
)

abp_model = IntraoperativeHypotensionModel(
    abp_resnet=abp_resnet
)

In [7]:
# Load data from google drive location
import pickle

case_list_file = open(r'cases_with_positive_samples.pkl', 'rb')
case_list = pickle.load(case_list_file)
case_list_file.close()

train_cases = case_list[0:80]
eval_cases = case_list[80:100]

train_data = get_data(3, from_dir='./drive/MyDrive/CS598/Final_Project/Case Set/', case_ids=train_cases)
eval_data = get_data(3, from_dir='./drive/MyDrive/CS598/Final_Project/Case Set/', case_ids=eval_cases)

Getting track data for case: 300
Statistics for case: 300, 590 total valid samples, 42 positive samples
Getting track data for case: 191
Statistics for case: 191, 691 total valid samples, 81 positive samples
Getting track data for case: 52
Statistics for case: 52, 1292 total valid samples, 130 positive samples
Getting track data for case: 233
Statistics for case: 233, 958 total valid samples, 47 positive samples
Getting track data for case: 66
Statistics for case: 66, 741 total valid samples, 7 positive samples
Getting track data for case: 149
Statistics for case: 149, 959 total valid samples, 4 positive samples
Getting track data for case: 166
Statistics for case: 166, 1466 total valid samples, 69 positive samples
Getting track data for case: 142
Statistics for case: 142, 1052 total valid samples, 12 positive samples
Getting track data for case: 287
Statistics for case: 287, 657 total valid samples, 9 positive samples
Getting track data for case: 270
Statistics for case: 270, 184 tota

<ipython-input-2-803fb0c24a94>:200: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  abps = FloatTensor(abps)
<ipython-input-2-803fb0c24a94>:203: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  hypotension_event_bools = BoolTensor(hypotension_event_bools).float()


Statistics for case: 327, 1725 total valid samples, 97 positive samples
Getting track data for case: 375
Statistics for case: 375, 1395 total valid samples, 391 positive samples
Getting track data for case: 330
Statistics for case: 330, 572 total valid samples, 12 positive samples
Getting track data for case: 370
Statistics for case: 370, 329 total valid samples, 32 positive samples
Getting track data for case: 371
Statistics for case: 371, 683 total valid samples, 6 positive samples
Getting track data for case: 390
Statistics for case: 390, 1841 total valid samples, 24 positive samples
Getting track data for case: 349
Statistics for case: 349, 856 total valid samples, 464 positive samples
Getting track data for case: 337
Statistics for case: 337, 592 total valid samples, 45 positive samples
Getting track data for case: 397
Statistics for case: 397, 1486 total valid samples, 451 positive samples
Getting track data for case: 348
Statistics for case: 348, 820 total valid samples, 7 posit

In [8]:
train(
    abp_model,
    train_data,
    eval_data,
    batch_size=256,
    epochs=100,
    model_title='abp_overnight_2',
    output_dir='./drive/MyDrive/CS598/Final_Project/model_checkpoints'
)

     Epoch #1
    Train data metrics:
        AUROC: 0.8635484532278495
        AUPRC: 0.37875503301620483
        Sensitivity: 0.0
        Specificity: 1.0
    Test data metrics:
        AUROC: 0.8210888195989321
        AUPRC: 0.327072411775589
        Sensitivity: 0.0
        Specificity: 1.0
     Epoch #2
    Train data metrics:
        AUROC: 0.8426890742011851
        AUPRC: 0.3435790538787842
        Sensitivity: 0.0
        Specificity: 1.0
    Test data metrics:
        AUROC: 0.80134330475596
        AUPRC: 0.29475289583206177
        Sensitivity: 0.0
        Specificity: 1.0
Mean loss: 0.10138352284396904
     Epoch #3
    Train data metrics:
        AUROC: 0.823436967043739
        AUPRC: 0.30354830622673035
        Sensitivity: 0.0018921475857496262
        Specificity: 0.9997585415840149
    Test data metrics:
        AUROC: 0.780240673132165
        AUPRC: 0.2637244164943695
        Sensitivity: 0.006038086488842964
        Specificity: 0.9997901916503906
Mean loss: 0.09

KeyboardInterrupt: 

## References
Jo YY, Jang JH, Kwon Jm, Lee HC, Jung CW, et al. (2022) Predicting intraoperative hypotension using deep learning with waveforms of arterial blood pressure, electroencephalogram, and electrocardiogram: Retrospective study. PLOS ONE 17(8): e0272055. https://doi.org/10.1371/journal.pone.0272055

## Acknowledgements
* As mentioned in the introduction, this project leveraged the open [vitaldb dataset](https://vitaldb.net/dataset/), and without it would have been impossible in its current form.
* Significant inspiration was drawn from [vital db examples](https://github.com/vitaldb/examples)

# Appendix

## Extraneous code / helpful snippets

In [16]:
# SANITY CHECK
train_data = get_data(3, from_dir='./drive/MyDrive/CS598/Final_Project/Case Set/', case_ids=['19'])
eval_data = get_data(3, from_dir='./drive/MyDrive/CS598/Final_Project/Case Set/', case_ids=['117'])

train(
    abp_model,
    train_data,
    eval_data,
    batch_size=256,
    epochs=100,
    learning_rate=0.001,
    # model_title='abp_overnight_1',
    # output_dir='./drive/MyDrive/CS598/Final_Project/model_checkpoints'
)

     Epoch #1
    Train data metrics:
        AUROC: 0.8227687603136276
        AUPRC: 0.44258809089660645
        Sensitivity: 0.0
        Specificity: 1.0
    Test data metrics:
        AUROC: 0.8515895800106327
        AUPRC: 0.21628309786319733
        Sensitivity: 0.0
        Specificity: 1.0
     Epoch #2
    Train data metrics:
        AUROC: 0.885431331052293
        AUPRC: 0.474674791097641
        Sensitivity: 0.0
        Specificity: 1.0
    Test data metrics:
        AUROC: 0.7606804891015417
        AUPRC: 0.12077982723712921
        Sensitivity: 0.0
        Specificity: 1.0
Mean loss: 0.18166158348321915
     Epoch #3
    Train data metrics:
        AUROC: 0.8880720858466731
        AUPRC: 0.45104217529296875
        Sensitivity: 0.0
        Specificity: 1.0
    Test data metrics:
        AUROC: 0.7296119085592769
        AUPRC: 0.10224129259586334
        Sensitivity: 0.0
        Specificity: 1.0
Mean loss: 0.1624432945623994
     Epoch #4
    Train data metrics:
       

KeyboardInterrupt: 

In [1]:
# Show case stats
import pickle

stats = open(r'cases_with_positive_samples_statistics.pkl', 'rb')
stat_obj = pickle.load(stats)
stats.close()

print(stat_obj)

{'1': {'positive': tensor(71.), 'negative': tensor(703.)}, '10': {'positive': tensor(111.), 'negative': tensor(1630.)}, '12': {'positive': tensor(319.), 'negative': tensor(1219.)}, '13': {'positive': tensor(31.), 'negative': tensor(498.)}, '16': {'positive': tensor(11.), 'negative': tensor(1031.)}, '17': {'positive': tensor(77.), 'negative': tensor(1648.)}, '19': {'positive': tensor(199.), 'negative': tensor(2156.)}, '20': {'positive': tensor(2.), 'negative': tensor(2167.)}, '24': {'positive': tensor(18.), 'negative': tensor(425.)}, '25': {'positive': tensor(9.), 'negative': tensor(1184.)}, '27': {'positive': tensor(14.), 'negative': tensor(1473.)}, '43': {'positive': tensor(208.), 'negative': tensor(949.)}, '49': {'positive': tensor(2.), 'negative': tensor(604.)}, '50': {'positive': tensor(23.), 'negative': tensor(1300.)}, '52': {'positive': tensor(130.), 'negative': tensor(1162.)}, '55': {'positive': tensor(13.), 'negative': tensor(1551.)}, '58': {'positive': tensor(291.), 'negative'

In [ ]:
import pickle

# Load dataset list and download files to google drive location
case_list_file = open(r'cases_with_positive_samples.pkl', 'rb')
case_list = pickle.load(case_list_file)
case_list_file.close()

for case_id in case_list:
  _download_vital_file(case_id)

!rm -rf ./drive/MyDrive/CS598/Final\ Project/Case\ Set
!mkdir ./drive/MyDrive/CS598/Final\ Project/Case\ Set
!mv ./*.vital ./drive/MyDrive/CS598/Final\ Project/Case\ Set/

In [ ]:
# 15000 total samples should be used for training, 1000 for validation
train_dataset = [
    full_data[0][0:15000].detach().clone(),
    full_data[1].detach().clone(),
    full_data[2].detach().clone(),
    full_data[3][0:15000].detach().clone()
]

eval_dataset = [
    full_data[0][15000:16000].detach().clone(),
    full_data[1].detach().clone(),
    full_data[2].detach().clone(),
    full_data[3][15000:16000].detach().clone()
]

full_data = None

train(abp_model, train_dataset, eval_dataset, batch_size=256, epochs=100)

In [ ]:
!mv ./abp_eval_data_set.pkl ./drive/MyDrive/CS598/Final\ Project/ABP_Datasets/
!mv ./abp_train_data_set.pkl ./drive/MyDrive/CS598/Final\ Project/ABP_Datasets/

In [ ]:
# Read data from ABP dataset at 3-minute before event

import pickle

# Load dataset list and import data 10 cases at a time
case_list_file = open(r'cases_with_positive_samples.pkl', 'rb')
case_list = pickle.load(case_list_file)
case_list_file.close()

data_blocks = []
!rm -rf ./load_now
!mkdir ./load_now

i = 0
for case_id in case_list:
  !cp ./abp_only_cases/{case_id}.vital ./load_now/
  i += 1
  if i == 10:
    # Cut a block
    block = get_data(3, from_dir='./load_now')

    processed = _process_loaded_abp_data(block)

    data_blocks.append(processed)
    !rm ./load_now/*.vital

    i = 0


In [ ]:
# Pickle the data blocks so we can easily load them later and not have to re-generate them
afile = open(r'abp_train_data_sets.pkl', 'wb')
pickle.dump(data_blocks, afile)
afile.close()
data_blocks = None

In [ ]:
# Load the data from the pickle file
data_file = open(r'abp_train_data_sets.pkl', 'rb')
data_blocks = pickle.load(data_file)
data_file.close()

# Split into train and eval sets
eval_dataset = data_blocks[-1]
train_dataset = data_blocks[0:-1]

data_blocks = None # GC

# Train block by block
for idx in range(len(train_dataset)):
  print(f'Training on data block {idx}!!!!')
  train(abp_model, train_dataset[idx], eval_dataset, batch_size=32, epochs=11)

In [ ]:
# # Get exact number of negative samples to match positive ones
# abp_neg = abp_neg[0:167]

# # Concat and shuffle together
# all_abp = torch.concat([abp_neg, abp_pos])
# all_labels = torch.concat([torch.zeros([abp_neg.size()[0]]), torch.ones([abp_pos.size()[0]])])
# shuffled = shuffle_tensors([all_abp, all_labels])

# train_set = [
#     shuffled[0],
#     torch.Tensor([]),
#     torch.Tensor([]),
#     shuffled[1]
# ]

# test_set = [
#     shuffled[0].detach().clone(),
#     torch.Tensor([]),
#     torch.Tensor([]),
#     shuffled[1].detach().clone()
# ]

NameError: name 'abp_neg' is not defined

In [ ]:
# Obtain ABP data to train on
#test_set = get_data(3, max_num_samples=1000)

# Get cases with good data
_download_vital_file('819')
_download_vital_file('3704')

all_data = get_data(3, from_dir='.')

Getting track data for case: 3704
Statistics for case: 3704, 1028 total valid samples, 94 positive samples
Getting track data for case: 819
Statistics for case: 819, 737 total valid samples, 73 positive samples
Max not reached but all available cases exhausted.  


<ipython-input-2-6f2b2df1b781>:194: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  abps = FloatTensor(abps)
<ipython-input-2-6f2b2df1b781>:197: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  hypotension_event_bools = BoolTensor(hypotension_event_bools).float()


In [ ]:
# Create a good balanced dataset

pos_indices = torch.nonzero(all_data[3] > 0.5).squeeze(-1)
neg_indices = torch.nonzero(all_data[3] < 0.5).squeeze(-1)
abp_pos = all_data[0].detach().clone()[pos_indices]
abp_neg = all_data[0].detach().clone()[neg_indices]

# Get exact number of negative samples to match positive ones
abp_neg = abp_neg[0:167]

# Concat and shuffle together
all_abp = torch.concat([abp_neg, abp_pos])
all_labels = torch.concat([torch.zeros([abp_neg.size()[0]]), torch.ones([abp_pos.size()[0]])])
shuffled = shuffle_tensors([all_abp, all_labels])

train_set = [
    shuffled[0],
    torch.Tensor([]),
    torch.Tensor([]),
    shuffled[1]
]

test_set = [
    shuffled[0].detach().clone(),
    torch.Tensor([]),
    torch.Tensor([]),
    shuffled[1].detach().clone()
]


In [ ]:
# Train the model on ABP data

# train_set = [
#     all_data[0][0:4000].unsqueeze(1),
#     all_data[1][0:4000].unsqueeze(1),
#     all_data[2][0:4000].unsqueeze(1),
#     all_data[3][0:4000]
# ]

# test_set = [
#     all_data[0][4000:].unsqueeze(1),
#     all_data[1][4000:].unsqueeze(1),
#     all_data[2][4000:].unsqueeze(1),
#     all_data[3][4000:]
# ]

# train(abp_model, train_set, test_set, batch_size=40, epochs=100, learning_rate=0.0001)

# TRAIN ON ONLY CASE 819

# new_train_set = (
#     train_set[0].detach().clone(),
#     train_set[1].detach().clone(),
#     train_set[2].detach().clone(),
#     torch.ones(train_set[3].size(), dtype=torch.float)
# )

# new_test_set = (
#     train_set[0].detach().clone(),
#     train_set[1].detach().clone(),
#     train_set[2].detach().clone(),
#     torch.ones(train_set[3].size(), dtype=torch.float)
# )

train(abp_model, train_set, test_set, batch_size=40, epochs=100)

# TEST WITH RANDOM DATA
# sample_size = 400
# train_set_r = [
#     torch.randn([sample_size, 1, 30000]),
#     torch.randn([sample_size, 1, 30000]),
#     torch.randn([sample_size, 1, 30000]),
#     torch.where(torch.rand([sample_size]) > 0.5, 1.0, 0.0),
# ]

# train(abp_model, train_set_r, train_set_r, batch_size=40, epochs=3)


     Epoch #1
	Batch 0 loss: 0.6892008185386658
	Batch 1 loss: 0.7042883038520813
	Batch 2 loss: 0.6887048482894897
	Batch 3 loss: 0.6910030245780945
	Batch 4 loss: 0.6846761703491211
	Batch 5 loss: 0.6989827752113342
	Batch 6 loss: 0.6940107345581055
	Batch 7 loss: 0.697780966758728
	Batch 8 loss: 0.6829289197921753
y_hat_long sum: 40, target_long sum: 22
y_hat_long sum: 40, target_long sum: 16
y_hat_long sum: 40, target_long sum: 22
y_hat_long sum: 40, target_long sum: 21
y_hat_long sum: 40, target_long sum: 24
y_hat_long sum: 40, target_long sum: 17
y_hat_long sum: 40, target_long sum: 19
y_hat_long sum: 40, target_long sum: 17
y_hat_long sum: 14, target_long sum: 9
    Train data metrics:
        AUROC: 0.9068835883725715
        AUPRC: 0.9036766886711121
        Sensitivity: 1.0
        Specificity: 0.0
y_hat_long sum: 40, target_long sum: 22
y_hat_long sum: 40, target_long sum: 16
y_hat_long sum: 40, target_long sum: 22
y_hat_long sum: 40, target_long sum: 21
y_hat_long sum: 40, 

KeyboardInterrupt: 

In [ ]:
import pickle

# Get cases and find their data counts
candy_cases = _get_candidate_cases()
left_off = rare_candy_cases[-1]
for case_id in candy_cases:
  if int(case_id) <= int(left_off):
    continue

  # Download the case
  _download_vital_file(case_id)

  # Load the case
  all_data = get_data(3, from_dir='.')

  # Analyze positive vs negative ratio
  length = len(torch.flatten(all_data[3]))
  if length > 0:
    pos = torch.sum(all_data[3])
    if pos > 1:
      rare_candy_cases.append(case_id)
      case_distribution[case_id] = { 'positive': pos, 'negative': (length - pos) }

  # Remove the case file
  !rm ./*.vital


Getting track data for case: 152
Statistics for case: 152, 428 total valid samples, 3 positive samples
Getting track data for case: 156
Statistics for case: 156, 0 total valid samples, 0 positive samples
Max not reached but all available cases exhausted.  


<ipython-input-2-6f2b2df1b781>:197: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  hypotension_event_bools = BoolTensor(hypotension_event_bools).float()


Getting track data for case: 153
Statistics for case: 153, 603 total valid samples, 6 positive samples
Max not reached but all available cases exhausted.  


<ipython-input-2-6f2b2df1b781>:197: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  hypotension_event_bools = BoolTensor(hypotension_event_bools).float()


Getting track data for case: 156
Statistics for case: 156, 2379 total valid samples, 0 positive samples
Max not reached but all available cases exhausted.  


...

<ipython-input-2-6f2b2df1b781>:197: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  hypotension_event_bools = BoolTensor(hypotension_event_bools).float()


Getting track data for case: 418
Statistics for case: 418, 1567 total valid samples, 33 positive samples
Max not reached but all available cases exhausted.  


<ipython-input-2-6f2b2df1b781>:197: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  hypotension_event_bools = BoolTensor(hypotension_event_bools).float()


Getting track data for case: 419


KeyboardInterrupt: 

In [ ]:
# pickle the results of case analysis
print(rare_candy_cases)

afile = open(r'cases_with_positive_samples.pkl', 'wb')
pickle.dump(rare_candy_cases, afile)
afile.close()

bfile = open(r'cases_with_positive_samples_statistics.pkl', 'wb')
pickle.dump(case_distribution, bfile)
bfile.close()

['1', '10', '12', '13', '16', '17', '19', '20', '24', '25', '27', '43', '49', '50', '52', '55', '58', '60', '61', '64', '66', '75', '79', '83', '84', '87', '92', '93', '94', '96', '97', '104', '105', '108', '111', '112', '116', '117', '118', '124', '135', '142', '143', '146', '148', '149', '152', '153', '161', '163', '166', '181', '183', '184', '191', '195', '197', '203', '208', '210', '233', '234', '236', '241', '244', '251', '252', '256', '261', '263', '266', '270', '272', '287', '293', '295', '300', '304', '309', '318', '323', '325', '327', '330', '337', '343', '348', '349', '354', '359', '363', '370', '371', '375', '382', '384', '387', '388', '390', '397', '406', '409', '415', '416', '417', '418']


In [ ]:
print(len(rare_candy_cases))

106


In [ ]:
# Put together model as the paper describes (with all resnets)
abp_resnet = WaveformResNet(
    input_shape=30000,
    output_size=32,
    data_type='abp'
)

ecg_resnet = WaveformResNet(
    input_shape=30000,
    output_size=32,
    data_type='ecg'
)

eeg_resnet = WaveformResNet(
    input_shape=7680,
    output_size=32,
    data_type='eeg'
)

model = IntraoperativeHypotensionModel(
    abp_resnet=abp_resnet,
    ecg_resnet=ecg_resnet,
    eeg_resnet=eeg_resnet
)